## Jobs Handling Example
Show the functions relates to the stopping of Jobs.

In [1]:
import asyncio
import time

from decanter import core
from decanter.core.core_api import TrainInput, PredictInput, SetupInput
from decanter.core.jobs import DataUpload, Experiment
from decanter.core.extra import CoreStatus
from decanter.core.enums.algorithms import Algo
from decanter.core.enums.evaluators import Evaluator

In [14]:
core.enable_default_logger()

# The Main object handles the calling of Decanter's API.
# Create connection to Decanter server, and set up basic settings.
# set the username, password, host
client = core.CoreClient(username=???, password=???, host=???)

In [3]:
train_file_path = 'data/train.csv'
test_file_path = 'data/test.csv'
train_file = open(train_file_path , 'rb')
test_file = open(test_file_path , 'rb')

## Job.stop() in diff status

In [15]:
train_data = client.upload(file=train_file, name='stop_when_running')

async def stop_when_running():
    while train_data.status == 'pending':
        await asyncio.sleep(3)
    train_data.stop()
    return

await stop_when_running()

Job failed if it is stopped in pending status.

In [16]:
train_data = DataUpload(name='stop_when_pending')
train_data.stop()
print(train_data.status)

Job remain failed if it is stopped in pending status.

In [17]:
train_data = DataUpload(name='stop_when_failed')
train_data.stop()
print(train_data.status)
train_data.stop()

## Test dependency
Stop DataUpload, then the Experiemnt and predict depends on the data will fail. Test data will keep upload till it's done.

In [18]:
train_file = open(train_file_path , 'rb')
test_file = open(test_file_path, 'rb')

train_data = client.upload(file=train_file, name='upload_train_data')
test_data = client.upload(file=test_file, name='upload_test_data')

setup_input = SetupInput(
        data = train_data,
        # data_id=train_data.id,
        data_source=train_data.accessor,
        data_columns=[
            {
                'id': 'Pclass',
                'data_type': 'categorical'
            }]
        )

train_data = client.setup(setup_input)
    
train_input = TrainInput(data=train_data, target='Survived', algos=[Algo.XGBoost], max_model=2, tolerance=0.9)
exp = client.train(train_input=train_input, select_model_by=Evaluator.auc, name='exp(data_failed)')

predict_input = PredictInput(data=test_data, experiment=exp)

# predict result
pred_res = client.predict(predict_input=predict_input, name='pred(exp_failed)')
    
async def stop():
    while train_data.status == CoreStatus.PENDING:
        await asyncio.sleep(3)
    
    train_data.stop()
    return

await stop()

## Test stop jobs list
Stop list of jobs, the jobs exclude in list will keep continue till done.

In [19]:
train_file = open(train_file_path , 'rb')

train_data = client.upload(file=train_file, name='upload_train_data')
train_input = TrainInput(data=train_data, target='Survived', algos=[Algo.XGBoost], max_model=2, tolerance=0.9)

exps = []
for i in range(5):
    exp = client.train(train_input=train_input, select_model_by=Evaluator.auc, name='exp_%d' % i)
    exps.append(exp)

time.sleep(4)
client.stop_jobs([exps[0], exps[2], exps[4]])


Get status of all the Jobs.

In [20]:
client.get_jobs_status()

## Test stop all jobs
Stop all jobs while data is done, all the experiment should be stopped

In [21]:
train_file = open(train_file_path , 'rb')

train_data = client.upload(file=train_file, name='upload_train_data')

train_input = TrainInput(data=train_data, target='Survived', algos=[Algo.XGBoost], max_model=2, tolerance=0.9)

exps = []
for i in range(5):
    exp = client.train(train_input=train_input, select_model_by=Evaluator.auc, name='exp_%d' % i)
    exps.append(exp)

async def stop():
    while train_data.status not in CoreStatus.DONE_STATUS:
        await asyncio.sleep(3)
    
    client.stop_all_jobs()
    return

await stop()


In [22]:
client.get_jobs_status()